In [1]:
import pandas as pd
print(pd.__version__)
df1 = pd.read_excel('학교별 교육편제단위 정보_20251002기준.xlsx', skiprows = 4 )
df2 = pd.read_excel('교육과정_대학(20250523).xlsx', skiprows = 8)

2.2.3


In [2]:
df1 = df1[df1['(대학)지역'] == '서울'].copy()
df1 = df1[(df1['대학구분'] == '대학') & (df1['학교구분'] == '대학교') & (df1['본분교'] == '본교') & (df1['(대학)지역'] == '서울')
          & (df1['주야간구분'] == '주간') & (df1['학과특성'] == '일반과정') & (df1['학과상태'] != '폐지') & (df1['수업연한'] == '4년')].copy() # 데이터 2070개 남음
df2 = df2[(df2['본분교명'] == '대학') & (df2['주야구분명'] == '주간') & (df2['학부특성명'] == '일반과정')].copy()

In [3]:
df2 = df2.rename(columns={
    "차수" : "조사차수",
    "본분교명": "대학구분",
    "학교구분" : "본분교",
    "학부·과(전공)코드": "학교별학과코드",
    "주야구분명" : "주야간구분",
    "학부특성명" : "학과특성",

})

In [4]:
df2.columns

Index(['조사년도', '조사차수', '학교코드', '학교명', '대학구분', '본분교', '단과대학코드', '단과대학명',
       '학교별학과코드', '학부·과(전공)명', '주야구분코드', '주야간구분', '학부특성코드', '학과특성', '학과상태',
       '교육과정', '이수구분', '학점', '교과목해설'],
      dtype='object')

In [6]:
# 1️⃣ 키 컬럼 전처리 (공백/괄호 제거, 타입 통일)
for df in [df1, df2]:
    df['학교코드'] = df['학교코드'].astype(str).str.strip()
    df['학교별학과코드'] = df['학교별학과코드'].astype(str).str.strip()
    df['학부·과(전공)명'] = (
        df['학부·과(전공)명']
        .astype(str)
        .str.replace(r'\s+', '', regex=True)       # 모든 공백 제거
        .str.replace(r'\(.*?\)', '', regex=True)   # 괄호 및 내용 제거 (선택)
    )

# 2️⃣ df2에서 필요한 열만 선택
attach_cols = ['교육과정','이수구분','학점','교과목해설']
df2_slim = df2[['학교코드','학교별학과코드','학부·과(전공)명'] + attach_cols].copy()

# 3️⃣ 조인
merged = pd.merge(
    df1,
    df2_slim,
    on=['학교코드','학교별학과코드','학부·과(전공)명'],
    how='left',      # 학과정보(df1)를 기준으로
    validate='1:m'   # 한 학과에 여러 과목이 붙는 구조
)

print(len(merged))
merged


85918


,조사년도,조사차수,학교명,학교코드,대학구분,본분교,학교구분,(대학)지역,단과대학명,단과대학코드,...,소계열분류,대학자체대계열,수업연한,학위과정,(학과)소재지,(학과)소재지\n(상세),교육과정,이수구분,학점,교과목해설
0,2025,2,감리교신학대학교,50,대학,본교,대학교,서울,신학대학,208,...,종교학,인문・사회계열,4년,학사,서울,서대문구,노인복지기관경영특론,NaN,NaN,NaN
1,2025,2,감리교신학대학교,50,대학,본교,대학교,서울,신학대학,208,...,종교학,인문・사회계열,4년,학사,서울,서대문구,인간관계론세미나,NaN,NaN,NaN
2,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,경영학,인문・사회계열,4년,학사,서울,강서구,한국어글쓰기와 표현1 (Korean writing and expression 1),전공필수,2.0,한국에 유학 온 외국인 학습자들이 자신의 생각과 감정을 표현하는 수업이다. (말하기...
3,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,경영학,인문・사회계열,4년,학사,서울,강서구,선형대수와편미분학 (Linear Algebra and Partial Calculus),전공선택,3.0,데이터를 분석하고 특히 인공지능을 학습하기 위해서는 수학적인 개념과 도구들을 이해해...
4,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,경영학,인문・사회계열,4년,학사,서울,강서구,경영통계학 (Business Statistics),전공필수,3.0,이 과정은 학생들이 비즈니스 분석을 수행할 수 있도록 준비시킨다. 비즈니스 환경 분...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85913,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,언어교육,인문・사회계열,4년,학사,서울,마포구,영문학개론,전공필수,3.0,이 과목은 미국문학개론 강좌로 볼 수 있다. 이 강좌는 학생들에게 아메리카 탄생의 ...
85914,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,언어교육,인문・사회계열,4년,학사,서울,마포구,영문법,전공필수,3.0,"전통적인 영어 문법을 실용적인 목적을 위하여 익히는 과정으로, 영어를 전공하는 학생..."
85915,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,언어교육,인문・사회계열,4년,학사,서울,마포구,시사영어,교양선택,2.0,This course focuses on learning English throug...
85916,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,언어교육,인문・사회계열,4년,학사,서울,마포구,미국문학,전공선택,3.0,"이 강좌는 미국문학 전반에 걸친 과목으로, 초기 미국문학부터 현대미국문학에 이르기까..."


In [ ]:
# merged.to_excel('C:\\Users\\heeyo\\Desktop\\KERIS_2종\\교육편제단위_표준분류체계 결합.xlsx', index=False)

## 교육과정 정제
- 뒤에 숫자로 나뉘어져 있는 과목 통일(초급중국어1, 초급중국어2 -> 초급중국어)
- 한영 표기 동시에 되어 있는 과목 숫자 제거 후 한글 표현으로 통일 (ex. 한국어글쓰기와 표현1 (Korean writing and expression 1))

In [14]:
import re

# 교육과정명이 숫자로 끝나는 행만 필터링
df_with_num = merged[merged['교육과정'].astype(str).str.contains(r'\d+$', regex=True)]

# 결과 확인
df_with_num[['학교명', '학부·과(전공)명', '교육과정']]

,학교명,학부·과(전공)명,교육과정
35,강서대학교,G2빅데이터경영학과,융합교과2
69,강서대학교,G2빅데이터경영학과,초급중국어2
70,강서대학교,G2빅데이터경영학과,초급중국어1
71,강서대학교,G2빅데이터경영학과,중급회계2
72,강서대학교,G2빅데이터경영학과,중급회계1
...,...,...,...
85272,홍익대학교,영어영문학과,Advanced English writing and speaking 2
85273,홍익대학교,영어영문학과,English writing and speaking 1
85274,홍익대학교,영어영문학과,English writing and speaking 2
85365,홍익대학교,동양화과,현대수묵화2


In [ ]:
# df_with_num.to_excel('숫자로 끝나는 교육과정_뒷 숫자 제거해 과목별 표본 n을 집계할 예정.xlsx')

In [20]:
import re
import pandas as pd

# 1️⃣ 함수 정의
def clean_course_name(name):
    if pd.isna(name):
        return name
    name = str(name).strip()
    # 문자열 끝의 숫자, -숫자, .숫자 제거
    name = re.sub(r'\d+([\-\.]\d+)?$', '', name)
    # 남은 공백 제거
    name = re.sub(r'\s+$', '', name)
    return name

# 2️⃣ '교육과정' 열에 바로 적용 (덮어쓰기)
merged['교육과정'] = merged['교육과정'].apply(clean_course_name)

# 3️⃣ 결과 확인
merged

,조사년도,조사차수,학교명,학교코드,대학구분,본분교,학교구분,(대학)지역,단과대학명,단과대학코드,...,대학자체대계열,수업연한,학위과정,(학과)소재지,(학과)소재지\n(상세),교육과정,이수구분,학점,교과목해설,표본수 n
0,2025,2,감리교신학대학교,50,대학,본교,대학교,서울,신학대학,208,...,인문・사회계열,4년,학사,서울,서대문구,노인복지기관경영특론,NaN,NaN,NaN,1
1,2025,2,감리교신학대학교,50,대학,본교,대학교,서울,신학대학,208,...,인문・사회계열,4년,학사,서울,서대문구,인간관계론세미나,NaN,NaN,NaN,1
2,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,인문・사회계열,4년,학사,서울,강서구,한국어글쓰기와 표현1 (Korean writing and expression 1),전공필수,2.0,한국에 유학 온 외국인 학습자들이 자신의 생각과 감정을 표현하는 수업이다. (말하기...,3
3,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,인문・사회계열,4년,학사,서울,강서구,선형대수와편미분학 (Linear Algebra and Partial Calculus),전공선택,3.0,데이터를 분석하고 특히 인공지능을 학습하기 위해서는 수학적인 개념과 도구들을 이해해...,1
4,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,인문・사회계열,4년,학사,서울,강서구,경영통계학 (Business Statistics),전공필수,3.0,이 과정은 학생들이 비즈니스 분석을 수행할 수 있도록 준비시킨다. 비즈니스 환경 분...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85913,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,인문・사회계열,4년,학사,서울,마포구,영문학개론,전공필수,3.0,이 과목은 미국문학개론 강좌로 볼 수 있다. 이 강좌는 학생들에게 아메리카 탄생의 ...,11
85914,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,인문・사회계열,4년,학사,서울,마포구,영문법,전공필수,3.0,"전통적인 영어 문법을 실용적인 목적을 위하여 익히는 과정으로, 영어를 전공하는 학생...",10
85915,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,인문・사회계열,4년,학사,서울,마포구,시사영어,교양선택,2.0,This course focuses on learning English throug...,1
85916,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,인문・사회계열,4년,학사,서울,마포구,미국문학,전공선택,3.0,"이 강좌는 미국문학 전반에 걸친 과목으로, 초기 미국문학부터 현대미국문학에 이르기까...",1


In [29]:
merged[merged['교육과정'].str.contains('Korean writing and expression', na=False)]

,조사년도,조사차수,학교명,학교코드,대학구분,본분교,학교구분,(대학)지역,단과대학명,단과대학코드,...,대학자체대계열,수업연한,학위과정,(학과)소재지,(학과)소재지\n(상세),교육과정,이수구분,학점,교과목해설,표본수 n
2,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,인문・사회계열,4년,학사,서울,강서구,한국어글쓰기와 표현1 (Korean writing and expression 1),전공필수,2.0,한국에 유학 온 외국인 학습자들이 자신의 생각과 감정을 표현하는 수업이다. (말하기...,3
179,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,인문・사회계열,4년,학사,서울,강서구,한국어글쓰기와 표현1 (Korean writing and expression 1),전공필수,2.0,한국에 유학 온 외국인 학습자들이 자신의 생각과 감정을 표현하는 수업이다. (말하기...,3
395,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,인문・사회계열,4년,학사,서울,강서구,한국어글쓰기와 표현1 (Korean writing and expression 1),전공필수,2.0,한국에 유학 온 외국인 학습자들이 자신의 생각과 감정을 표현하는 수업이다. (말하기...,3


In [32]:
import re
import pandas as pd

# 우선 전체 과목 목록 확보
all_courses = merged['교육과정'].dropna().unique().tolist()

# 1️⃣ 영어명 → 한글명 매핑 사전 생성
#    (괄호가 포함된 항목에서 추출)
eng2kor = {}
for c in all_courses:
    if '(' in c and ')' in c:
        kor = re.sub(r'\(.*?\)', '', c).strip()
        inside = re.findall(r'\((.*?)\)', c)
        if inside:
            inside_text = inside[0].strip()
            # 영어가 포함되어 있으면 사전 등록
            if re.search(r'[A-Za-z]', inside_text):
                # 괄호 속 영어명만 남기기 (숫자 제거)
                eng = re.sub(r'\d+([\-\.]\d+)?$', '', inside_text).strip()
                eng2kor[eng.lower()] = kor

print("생성된 영어→한글 매핑 개수:", len(eng2kor))

# 2️⃣ 정제 함수 정의
def clean_course_name_v3(name):
    if pd.isna(name):
        return name
    name = str(name).strip()
    name = name.replace('（','(').replace('）',')')

    # (1) 중첩 괄호 모두 제거
    while re.search(r'\([^()]*\)', name):
        name = re.sub(r'\([^()]*\)', '', name)
    name = name.strip()

    # (2) 끝의 숫자/-.숫자 제거
    name = re.sub(r'\d+([\-\.]\d+)?$', '', name).strip()

    # (3) 영어→한글 자동 치환 (대소문자 무시)
    lower_name = name.lower()
    for eng, kor in eng2kor.items():
        if eng in lower_name:
            name = kor
            break

    # (4) 공백 정리
    name = re.sub(r'\s+', ' ', name).strip()
    return name

# 3️⃣ 적용
before = merged['교육과정'].copy()
merged['교육과정'] = merged['교육과정'].apply(clean_course_name_v3)

# 4️⃣ 변경 행 비교
changed = merged[before != merged['교육과정']][['학교명','학부·과(전공)명','교육과정']].copy()
changed['변경_전'] = before[before != merged['교육과정']]
print("변경된 행 수:", len(changed))
display(changed.head(10))


생성된 영어→한글 매핑 개수: 1134
변경된 행 수: 17391


,학교명,학부·과(전공)명,교육과정,변경_전
2,강서대학교,AI기반경영학과,한국어글쓰기와 표현,한국어글쓰기와 표현1 (Korean writing and expression 1)
3,강서대학교,AI기반경영학과,선형대수와편미분학,선형대수와편미분학 (Linear Algebra and Partial Calculus)
4,강서대학교,AI기반경영학과,경영통계학,경영통계학 (Business Statistics)
5,강서대학교,AI기반경영학과,객체지향프로그래밍,객체지향프로그래밍 (Object-oriented Programming)
6,강서대학교,AI기반경영학과,비즈니스한국어,비즈니스한국어2 (Business Korean2)
7,강서대학교,AI기반경영학과,인공지능윤리,인공지능윤리 (AI Ethics)
8,강서대학교,AI기반경영학과,데이터베이스개론,데이터베이스개론 (Introduction to Database)
9,강서대학교,AI기반경영학과,기계학습Ⅰ,기계학습Ⅰ(Machine LearningⅠ)
10,강서대학교,AI기반경영학과,기업가정신,기업가정신 (Entrepreneurship)
11,강서대학교,AI기반경영학과,재품생산및운영전략,재품생산및운영전략 (Production and operation strategy)


In [34]:
merged['표본수 n'] = (
    merged.groupby('교육과정')['교육과정'].transform('count').astype('Int64')
)

merged

,조사년도,조사차수,학교명,학교코드,대학구분,본분교,학교구분,(대학)지역,단과대학명,단과대학코드,...,대학자체대계열,수업연한,학위과정,(학과)소재지,(학과)소재지\n(상세),교육과정,이수구분,학점,교과목해설,표본수 n
0,2025,2,감리교신학대학교,50,대학,본교,대학교,서울,신학대학,208,...,인문・사회계열,4년,학사,서울,서대문구,노인복지기관경영특론,NaN,NaN,NaN,1
1,2025,2,감리교신학대학교,50,대학,본교,대학교,서울,신학대학,208,...,인문・사회계열,4년,학사,서울,서대문구,인간관계론세미나,NaN,NaN,NaN,1
2,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,인문・사회계열,4년,학사,서울,강서구,한국어글쓰기와 표현,전공필수,2.0,한국에 유학 온 외국인 학습자들이 자신의 생각과 감정을 표현하는 수업이다. (말하기...,3
3,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,인문・사회계열,4년,학사,서울,강서구,선형대수와편미분학,전공선택,3.0,데이터를 분석하고 특히 인공지능을 학습하기 위해서는 수학적인 개념과 도구들을 이해해...,17
4,2025,2,강서대학교,79,대학,본교,대학교,서울,단과대구분없음,999,...,인문・사회계열,4년,학사,서울,강서구,경영통계학,전공필수,3.0,이 과정은 학생들이 비즈니스 분석을 수행할 수 있도록 준비시킨다. 비즈니스 환경 분...,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85913,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,인문・사회계열,4년,학사,서울,마포구,영문학개론,전공필수,3.0,이 과목은 미국문학개론 강좌로 볼 수 있다. 이 강좌는 학생들에게 아메리카 탄생의 ...,19
85914,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,인문・사회계열,4년,학사,서울,마포구,영문법,전공필수,3.0,"전통적인 영어 문법을 실용적인 목적을 위하여 익히는 과정으로, 영어를 전공하는 학생...",12
85915,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,인문・사회계열,4년,학사,서울,마포구,시사영어,교양선택,2.0,This course focuses on learning English throug...,2
85916,2025,2,홍익대학교,212,대학,본교,대학교,서울,사범대학,161,...,인문・사회계열,4년,학사,서울,마포구,미국문학,전공선택,3.0,"이 강좌는 미국문학 전반에 걸친 과목으로, 초기 미국문학부터 현대미국문학에 이르기까...",1


In [36]:
merged.to_excel('교육과정 정제 후 표본수 n 집계 완료(1)_20251010.xlsx')